# Energy Data PreProcessing

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats.stats import pearsonr

## Load Data

In [2]:
def prepare_data(df, column_name):
    """
    Processes BP data. column_name is desired name for this data
    """
    df = df.loc[~(df["country"].isnull() | df["country"].str.contains("Total") | df["country"].str.contains("OECD"))]
    df = df.set_index("country")
    df = df.stack().rename(column_name).reset_index().rename(columns={"level_1":"year"})
    df = df.loc[df["year"].astype(str).str.len() == 4]
    df.loc[:, "year"] = pd.to_numeric(df.loc[:,"year"])
    df.loc[:, column_name] = pd.to_numeric(df.loc[:,column_name])
    return df

In [3]:
BP_FILE = "../../raw_data/usage/bp-stats-review-2019-all-data.xlsx"

#### CO2

Units: Million tonnes of CO2

In [4]:
co2 = pd.read_excel(BP_FILE, "Carbon Dioxide Emissions", header=2)
co2.rename(columns={"Million tonnes of carbon dioxide":"country"}, inplace=True)
co2 = prepare_data(co2, "co2")

#### Total Energy

In [5]:
energy = pd.read_excel(BP_FILE, "Primary Energy Consumption", header=2)
energy.rename(columns={"Million tonnes oil equivalent":"country"}, inplace=True)
energy = prepare_data(energy, "energy_consumption")

#### Total Renewables

In [6]:
renew = pd.read_excel(BP_FILE, "Renewables - Mtoe", header=2)
renew.rename(columns={"Million tonnes oil equivalent":"country"}, inplace=True)
renew = prepare_data(renew, "renewable_generation")

#### Solar

In [7]:
solar = pd.read_excel(BP_FILE, "Solar Generation - TWh", header=2)
solar.rename(columns={"Terawatt-hours":"country"}, inplace=True)
solar = prepare_data(solar, "solar_generation")

#### Wind

In [8]:
wind = pd.read_excel(BP_FILE, "Wind Generation - TWh ", header=2)
wind.rename(columns={"Terawatt-hours":"country"}, inplace=True)
wind = prepare_data(wind, "wind_generation")

#### Hydroelectricity

In [9]:
hydro = pd.read_excel(BP_FILE, "Hydro Generation - TWh", header=2)
hydro.rename(columns={"Terawatt-hours":"country"}, inplace=True)
hydro = prepare_data(hydro, "hydro_generation")

#### Geothermal, biomass, other

In [10]:
other = pd.read_excel(BP_FILE, "Geo Biomass Other - TWh", header=2)
other.rename(columns={"Terawatt-hours":"country"}, inplace=True)
other = prepare_data(other, "geo_bio_other_generation")

### Combine

In [11]:
df = (co2.merge(energy, on=["country","year"], how="left")
      .merge(renew, on=["country", "year"], how="left")
      .merge(solar, on=["country", "year"], how="left")
      .merge(wind, on=["country", "year"], how="left")
      .merge(hydro, on=["country", "year"], how="left")
      .merge(other, on=["country", "year"], how="left")
     )

#### Changes units from mtoe to TWh

1 mtoe = 4.4 TWh 

In [12]:
df["energy_consumption"] = df["energy_consumption"] * 4.4
df["renewable_generation"] = df["renewable_generation"] * 4.4

total renewable energy in BP does not include hydroelectricity, so add hydro energy generated

In [13]:
df["renewable_generation"] = df["renewable_generation"] + df['hydro_generation']

computer ratio statistics

In [14]:
df["renew_energy_ratio"] = (df["renewable_generation"]) / df["energy_consumption"]
df["emission_power_ratio"] = df["co2"]/df["energy_consumption"]

## Shape Data

#### Clean country names

In [15]:
df["country"] = df["country"].replace({"US": "United States", "Iran": "Iran (Islamic Republic of)",
                                       "South Korea": "Korea, Republic of", "China Hong Kong SAR": "Hong Kong",
                                       "Trinidad & Tobago": "Trinidad and Tobago", "Vietnam": "Viet Nam",
                                       "Russian Federation": "Russia"
                                      })

#### Shape Files

In [16]:
countries = gpd.read_file("../../raw_data/shape_files/TM_WORLD_BORDERS-0.3.shp")

In [17]:
regions = pd.read_csv("../../raw_data/regions.csv")
countries= countries.merge(regions, left_on="REGION", right_on="code", how="left")
countries["region"].fillna("Other", inplace=True)

In [18]:
countries["region"].value_counts()

Africa      57
Americas    53
Europe      51
Asia        50
Oceania     25
Other       10
Name: region, dtype: int64

In [19]:
countries.loc[:, ["NAME", "region"]].to_csv("../regions_mapping.csv", index=False)

In [20]:
df.loc[~df["country"].isin(countries["NAME"])]["country"].value_counts()

Other Northern Africa                54
Western Africa                       54
Other Asia Pacific                   54
Other Middle East                    54
Other Europe                         54
Other South America                  54
Other Caribbean                      54
                 European Union #    54
Other Southern Africa                54
Eastern Africa                       54
Middle Africa                        54
Central America                      54
Other CIS                            34
North Macedonia                      29
USSR                                 20
Name: country, dtype: int64

In [21]:
df = countries.loc[:, ["ISO3", "NAME", "AREA", "POP2005", "LON", "LAT"]].merge(df, left_on="NAME", right_on="country", how="left")

In [22]:
df.head()

,ISO3,NAME,AREA,POP2005,LON,LAT,country,year,co2,energy_consumption,renewable_generation,solar_generation,wind_generation,hydro_generation,geo_bio_other_generation,renew_energy_ratio,emission_power_ratio
0,ATG,Antigua and Barbuda,44,83039,-61.783,17.078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DZA,Algeria,238174,32854159,2.632,28.163,Algeria,1965.0,5.568753,9.339093,0.393327,0.0,0.0,0.393327,0.0,0.042116,0.596284
2,DZA,Algeria,238174,32854159,2.632,28.163,Algeria,1966.0,6.867506,11.250250,0.362391,0.0,0.0,0.362391,0.0,0.032212,0.610431
3,DZA,Algeria,238174,32854159,2.632,28.163,Algeria,1967.0,6.369307,10.633018,0.406585,0.0,0.0,0.406585,0.0,0.038238,0.599012
4,DZA,Algeria,238174,32854159,2.632,28.163,Algeria,1968.0,6.771137,11.405035,0.556844,0.0,0.0,0.556844,0.0,0.048824,0.593697


In [23]:
df.to_csv("../usage/processed.csv", index=False)

### Interactive datasets

Generate 2018 dataset for top countries ranked by renewable energy generated for easy use in d3

In [30]:
(df.pivot(index="year", columns="NAME", values=["renew_energy_ratio"])
              .iloc[-1].dropna().sort_values(ascending=False).to_frame().droplevel(0).T
    .to_csv("../usage/top_countries.csv", index=False))